In [1]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig

### Read network and graph

In [2]:
network = Network("data/porto-bbox/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

Nodes 31588 edges 72785


### Precompute UBODT

In [3]:
# Can be skipped if you already generated an ubodt file
ubodt_gen = UBODTGenAlgorithm(network,graph)
# The delta is defined as 3 km approximately. 0.03 degrees. 
status = ubodt_gen.generate_ubodt("data/ubodt.txt", 0.03, binary=False, use_omp=True)
# Binary is faster for both IO and precomputation
# status = ubodt_gen.generate_ubodt("data/ubodt.txt", 0.03, binary=True, use_omp=True)
print status

Status: success
Time takes 47.825 seconds



### Create FMM model

In [4]:
### Read UBODT
ubodt = UBODT.read_ubodt_csv("data/ubodt.txt")
### Create FMM model
fmm_model = FastMapMatch(network,graph,ubodt)

In [5]:
k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching for wkt

In [6]:
# wkt = "LINESTRING(18.05522129697668 59.335820514008,18.050801016520136 59.33614881818502)"
wkt='LINESTRING(-8.618643 41.141412,-8.618499 41.141376)'
result = fmm_model.match_wkt(wkt,fmm_config)
### Print map matching result
print "Opath ",list(result.opath)
print "Cpath ",list(result.cpath)
print "WKT ",result.mgeom.export_wkt()

Opath  [1095, 1095]
Cpath  [1095]
WKT  LINESTRING(-8.6186233 41.141456,-8.6184903 41.141395)


### Create stmatch model

In [7]:
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

In [8]:
stmatch_model = STMATCH(network,graph)

In [9]:
k = 4
gps_error = 0.0005
radius = 0.003
vmax = 0.0003
factor = 1.5
stmatch_config = STMATCHConfig(k, radius, gps_error, vmax, factor)

In [10]:
# wkt = "LINESTRING(18.05522129697668 59.335820514008,18.050801016520136 59.33614881818502,18.04796860380041 59.33905964310512,18.046037413309687 59.341029232754444,18.049084402750605 59.34262670491522,18.052775122355097 59.34328317853977,18.056809164713496 59.34273611806721,18.059298254679316 59.34006633652867)"
wkt='LINESTRING(-8.618643 41.141412,-8.618499 41.141376,-8.620326 41.14251,-8.622153 41.143815,-8.623953 41.144373,-8.62668 41.144778,-8.627373 41.144697,-8.630226 41.14521,-8.632746 41.14692,-8.631738 41.148225,-8.629938 41.150385,-8.62911 41.151213,-8.629128 41.15124,-8.628786 41.152203,-8.628687 41.152374,-8.628759 41.152518,-8.630838 41.15268,-8.632323 41.153022,-8.631144 41.154489,-8.630829 41.154507,-8.630829 41.154516,-8.630829 41.154498,-8.630838 41.154489)'
result = fmm_model.match_wkt(wkt,fmm_config)
### Print map matching result
print "Opath ",list(result.opath)
print "Cpath ",list(result.cpath)
# print "spdist",list(result.spdist)
print "WKT ",result.mgeom.export_wkt()

Opath  [1095, 1095, 2732, 2732, 1093, 1093, 698, 698, 696, 8687, 17386, 3607, 3607, 2723, 2723, 1337, 1339, 41290, 1435, 1434, 1434, 1434, 1434]
Cpath  [1095, 2732, 1093, 698, 8704, 62550, 8703, 696, 8687, 17386, 3607, 5509, 2723, 1337, 1339, 55275, 41288, 41290, 61207, 58489, 1435, 1434]
WKT  LINESTRING(-8.6186233 41.141456,-8.6183463 41.14133,-8.6182629 41.141292,-8.6174342 41.140917,-8.6182629 41.141292,-8.6183463 41.14133,-8.6187144 41.141497,-8.6188491 41.141582,-8.6198235 41.142218,-8.6200853 41.142389,-8.6202318 41.14249,-8.6203556 41.142575,-8.6209478 41.143157,-8.6212837 41.143398,-8.6213339 41.143434,-8.6216294 41.143598,-8.6216904 41.143624,-8.6220438 41.143771,-8.6225335 41.143949,-8.6231598 41.144138,-8.6235745 41.144265,-8.6243174 41.144494,-8.6247666 41.144632,-8.6250934 41.144735,-8.6254106 41.144793,-8.6266799 41.14476,-8.6267551 41.144759,-8.6268987 41.144728,-8.627037 41.144713,-8.6271975 41.1447,-8.6273413 41.144688,-8.6274999 41.144678,-8.6276791 41.14467,-8.627880

### Match GPS data stored in a file¶

In [11]:
from fmm import GPSConfig,ResultConfig

In [12]:
import pandas as pd
import csv
raw_trips = pd.read_csv('data/train-1000.csv')
from ast import literal_eval
raw_trajectories = [literal_eval(raw_trips['POLYLINE'].iloc[i]) for i in range(len(raw_trips.index))]
# ids=[raw_trips['TRIP_ID'].iloc[i] for i in range(len(raw_trips.index))]
ids=[i+1 for i in range(len(raw_trips.index))]
# print(raw_trajectories[0])
print(len(ids),len(raw_trajectories))
trajectories=[[str(indx[0])+' '+str(indx[1]) for indx in trajectory] for trajectory in raw_trajectories  ]
# print(trajectories[0])

trips=[['id;geom']]
for trip,index in zip(trajectories,ids):
    if len(trip)>0:
        one_trip=[str(index)+';LINESTRING('+trip[0]]
        if len(trip)>1:
            for indx in trip[1:-1]:
                one_trip.append(indx)

        one_trip.append(trip[-1]+')')
        trips.append(one_trip)
print(trips[1])
with open('data/trips.csv','w') as df:
    writer=csv.writer(df)
    writer=writer.writerows(trips[:])
df.close()
# df=pd.DataFrame({'id':ids,'geom':trips})
# df.to_csv('data/trips.csv',index=0)

(1000, 1000)
['1;LINESTRING(-8.618643 41.141412', '-8.618499 41.141376', '-8.620326 41.14251', '-8.622153 41.143815', '-8.623953 41.144373', '-8.62668 41.144778', '-8.627373 41.144697', '-8.630226 41.14521', '-8.632746 41.14692', '-8.631738 41.148225', '-8.629938 41.150385', '-8.62911 41.151213', '-8.629128 41.15124', '-8.628786 41.152203', '-8.628687 41.152374', '-8.628759 41.152518', '-8.630838 41.15268', '-8.632323 41.153022', '-8.631144 41.154489', '-8.630829 41.154507', '-8.630829 41.154516', '-8.630829 41.154498', '-8.630838 41.154489)']


In [13]:
# Define input data configuration
input_config = GPSConfig()
input_config.file = "data/trips.csv"
input_config.id = "id"
print input_config.to_string()

gps file : data/trips.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : x
y column : y
GPS point : false



In [18]:
# Define output configuration
result_config = ResultConfig()
result_config.file = "data/mr.csv"
result_config.output_config.write_opath = True
# result_config.output_config.write_duration = True
# result_config.output_config.write_spdlist = True
result_config.output_config.write_mgeom = True
result_config.output_config.write_pgeom = True

print result_config.to_string()


Result file : data/mr.csv
Output fields: opath pgeom cpath mgeom 


In [19]:
status = fmm_model.match_gps_file(input_config, result_config, fmm_config, use_omp = True)
print status

Status: success
Time takes 1.516 seconds
Total points 47337 matched 44086
Total trajectories 999 matched 948
Map match percentage 0.931322
Map match speed 29080.5 points/s 



In [20]:
status = stmatch_model.match_gps_file(input_config, result_config, stmatch_config, use_omp = True)
print status

Status: success
Time takes 5.144 seconds
Total points 47337 matched 46192
Map match speed 8979.78 points/s 

